In [1]:
import numpy as np
import os
import sys

import wave
import copy
import math

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Merge,Bidirectional
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize


from features import *
from helper import *


Using TensorFlow backend.


In [2]:

code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = code_path + "/../data/sessions/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000


In [3]:
import pickle
with open(data_path + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [7]:
def calculate_features(frames, freq, options):
    window_sec = 0.2
    window_n = int(freq * window_sec)

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)

    if st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1
        
        deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
        return deriv_st_f
    elif st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f

In [8]:
x_train_speech = []

counter = 0
for ses_mod in data2:
    if 'impro' not in ses_mod['id']:
        continue
    x_head = ses_mod['signal']
    st_features = calculate_features(x_head, framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
    x_train_speech.append( st_features.T )
    counter+=1
    if(counter%100==0):
        print(counter)
    
x_train_speech = np.array(x_train_speech)
x_train_speech.shape

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600


(2659, 100, 34)

In [9]:
def lstm_model(optimizer='Adadelta'):
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(100, 34)))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [10]:
model = lstm_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1120256   
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 2052      
_________________________________________________________________
activation_2 (Activation)    (None, 4)                 0         
Total params: 2,041,348
Trainable params: 2,041,348
Non-trainable params: 0
_________________________________________________________________


In [11]:
Y=[]
for ses_mod in data2:
    if 'impro' not in ses_mod['id']:
        continue
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(2659, 4)

In [12]:
hist = model.fit(x_train_speech, Y, 
                 batch_size=100, nb_epoch=60, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 2127 samples, validate on 532 samples
Epoch 1/60
2127/2127 [==============================] - 27s - loss: 1.3567 - acc: 0.3780 - val_loss: 1.2005 - val_acc: 0.5301
Epoch 2/60
2127/2127 [==============================] - 8s - loss: 1.3189 - acc: 0.3855 - val_loss: 1.1977 - val_acc: 0.5132
Epoch 3/60
2127/2127 [==============================] - 8s - loss: 1.3185 - acc: 0.3883 - val_loss: 1.2156 - val_acc: 0.5132
Epoch 4/60
2127/2127 [==============================] - 8s - loss: 1.3164 - acc: 0.3874 - val_loss: 1.1594 - val_acc: 0.5301
Epoch 5/60
2127/2127 [==============================] - 8s - loss: 1.3154 - acc: 0.3883 - val_loss: 1.1900 - val_acc: 0.5150
Epoch 6/60
2127/2127 [==============================] - 8s - loss: 1.3127 - acc: 0.3869 - val_loss: 1.1961 - val_acc: 0.5169
Epoch 7/60
2127/2127 [==============================] - 8s - loss: 1.3142 - acc: 0.3893 - val_loss: 1.2119 - val_acc: 0.5075
Epoch 8/60
2127/2127 [==============================] - 8s - loss: 1.3138 - a

In [13]:
hist = model.fit(x_train_speech, Y, 
                 batch_size=100, nb_epoch=15, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 2127 samples, validate on 532 samples
Epoch 1/15
2127/2127 [==============================] - 8s - loss: 0.9284 - acc: 0.6098 - val_loss: 1.0951 - val_acc: 0.5414
Epoch 2/15
2127/2127 [==============================] - 8s - loss: 0.9361 - acc: 0.5976 - val_loss: 1.0913 - val_acc: 0.5132
Epoch 3/15
2127/2127 [==============================] - 8s - loss: 0.9207 - acc: 0.6027 - val_loss: 1.0085 - val_acc: 0.6128
Epoch 4/15
2127/2127 [==============================] - 8s - loss: 0.9290 - acc: 0.5976 - val_loss: 0.9203 - val_acc: 0.6335
Epoch 5/15
2127/2127 [==============================] - 8s - loss: 0.9153 - acc: 0.6013 - val_loss: 1.4090 - val_acc: 0.4530
Epoch 6/15
2127/2127 [==============================] - 8s - loss: 0.9328 - acc: 0.5905 - val_loss: 1.0003 - val_acc: 0.5602
Epoch 7/15
2127/2127 [==============================] - 8s - loss: 0.8968 - acc: 0.6008 - val_loss: 1.2632 - val_acc: 0.4962
Epoch 8/15
2127/2127 [==============================] - 8s - loss: 0.9171 - ac

In [14]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent, _time_distributed_dense
from keras.engine import InputSpec

tfPrint = lambda d, T: tf.Print(input_=T, data=[T, tf.shape(T)], message=d)

class AttentionDecoder(Recurrent):

    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space

        references:
            Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio.
            "Neural machine translation by jointly learning to align and translate."
            arXiv preprint arXiv:1409.0473 (2014).
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = _time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)

        return super(AttentionDecoder, self).call(x)

    def get_initial_state(self, inputs):
        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1-zt)*stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [15]:
def attention_model(optimizer='Adadelta'):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(100, 34)))
    model.add(AttentionDecoder(128,128))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [16]:
model = attention_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 128)          83456     
_________________________________________________________________
AttentionDecoder (AttentionD (None, 100, 128)          246528    
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               6554112   
_________________________________________________________________
activation_3 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 2052      
_________________________________________________________________
activation_4 (Activation)    (None, 4)                 0         
Total para

In [17]:
hist = model.fit(x_train_speech, Y, 
                 batch_size=100, nb_epoch=75, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 2127 samples, validate on 532 samples
Epoch 1/75
2127/2127 [==============================] - 14s - loss: 1.3290 - acc: 0.3808 - val_loss: 1.1877 - val_acc: 0.5301
Epoch 2/75
2127/2127 [==============================] - 13s - loss: 1.3111 - acc: 0.3846 - val_loss: 1.1691 - val_acc: 0.5301
Epoch 3/75
2127/2127 [==============================] - 13s - loss: 1.2971 - acc: 0.3973 - val_loss: 1.2696 - val_acc: 0.3102
Epoch 4/75
2127/2127 [==============================] - 13s - loss: 1.2992 - acc: 0.3832 - val_loss: 1.2033 - val_acc: 0.5301
Epoch 5/75
2127/2127 [==============================] - 13s - loss: 1.2706 - acc: 0.4104 - val_loss: 1.1750 - val_acc: 0.5301
Epoch 6/75
2127/2127 [==============================] - 13s - loss: 1.2789 - acc: 0.4006 - val_loss: 1.1752 - val_acc: 0.5301
Epoch 7/75
2127/2127 [==============================] - 13s - loss: 1.2548 - acc: 0.4015 - val_loss: 1.1022 - val_acc: 0.5620
Epoch 8/75
2127/2127 [==============================] - 13s - loss: 1.1

In [18]:
counter = 0
for ses_mod in data2:
    if 'impro' not in ses_mod['id']:
        continue
    if (ses_mod['id'][:5]=="Ses05"):
        break
    counter+=1
counter

2034

In [22]:
xtrain_sp = x_train_speech[:2034]
xtest_sp = x_train_speech[2034:]
ytrain_sp = Y[:2034]
ytest_sp = Y[2034:]

In [23]:
def attention_model(optimizer='Adadelta'):
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(100, 34)))
    model.add(Bidirectional(AttentionDecoder(128,128)))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

model = attention_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 100, 256)          166912    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 100, 256)          689664    
_________________________________________________________________
flatten_3 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               13107712  
_________________________________________________________________
activation_7 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 2052      
_________________________________________________________________
activation_8 (Activation)    (None, 4)                 0         
Total para

In [24]:
hist = model.fit(xtrain_sp, ytrain_sp, 
                 batch_size=100, nb_epoch=100, verbose=1, shuffle = True, 
                 validation_data=(xtest_sp, ytest_sp))

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 2034 samples, validate on 625 samples
Epoch 1/100
2034/2034 [==============================] - 29s - loss: 1.3196 - acc: 0.3992 - val_loss: 1.2176 - val_acc: 0.4592
Epoch 2/100
2034/2034 [==============================] - 27s - loss: 1.3013 - acc: 0.3963 - val_loss: 1.2180 - val_acc: 0.4592
Epoch 3/100
2034/2034 [==============================] - 27s - loss: 1.2982 - acc: 0.3982 - val_loss: 1.2202 - val_acc: 0.4464
Epoch 4/100
2034/2034 [==============================] - 27s - loss: 1.2759 - acc: 0.4115 - val_loss: 1.1676 - val_acc: 0.4832
Epoch 5/100
2034/2034 [==============================] - 27s - loss: 1.2546 - acc: 0.4204 - val_loss: 1.2124 - val_acc: 0.4592
Epoch 6/100
2034/2034 [==============================] - 27s - loss: 1.2000 - acc: 0.4336 - val_loss: 1.2109 - val_acc: 0.3456
Epoch 7/100
2034/2034 [==============================] - 27s - loss: 1.1830 - acc: 0.4223 - val_loss: 1.0992 - val_acc: 0.4848
Epoch 8/100
2034/2034 [==============================] - 27s - l

In [34]:
data2[1800]['id']

'Ses02M_script03_1_M015'